In [14]:
import numpy as np
from scipy import sparse as ss
from scipy.sparse.linalg import LinearOperator, svds
from sklearn.utils.extmath import svd_flip
import pandas as pd
import datatable as dt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import manifold
from sklearn.metrics.pairwise import pairwise_distances
from umap import UMAP
from umap import umap_

import igraph as ig
import leidenalg
# import louvain as lv
from hdbscan import HDBSCAN

In [2]:
def read_raw(count_file, cell_file, gene_file):
    df = dt.fread(count_file, skip_to_line=2, header=False).to_pandas()
    cell_df = pd.read_csv(cell_file, header=None, names=["cell"], squeeze=True)
    gene_df = pd.read_csv(gene_file, header=None, names=["gene"], squeeze=True)
    df.columns = ["gene", "cell", "counts"]
    gene_num, cell_num, total_count = df.iloc[0]
    df.drop(index=0, inplace=True)
    df.cell -= 1
    df.gene -= 1
    X = ss.csr_matrix((df.counts, (df.cell, df.gene)), shape=(cell_num, gene_num), dtype='int16')

    df = pd.DataFrame.sparse.from_spmatrix(
        X, index=cell_df, columns=gene_df,
    ).astype("Sparse[int16]")
    df.sort_index(inplace=True)
    return df


def qc(
    df,
    cell_min_counts=0,
    cell_max_counts=np.inf,
    cell_min_genes=0,
    cell_max_genes=np.inf,
    gene_min_counts=0,
    gene_max_counts=np.inf,
    gene_min_cells=0,
    gene_max_cells=np.inf,
):
    cell_num, gene_num = df.shape
    for i in cell_min_genes, cell_max_genes:
        if (not np.isinf(i)) and isinstance(i, float):
            i = int(i * gene_num)
    for i in gene_min_cells, gene_max_cells:
        if (not np.isinf(i)) and isinstance(i, float):
            i = int(i * cell_num)

    binarized_df = df > 0
    cell_counts = df.sum(axis=1)
    cell_genes = binarized_df.sum(axis=1)

    gene_counts = df.sum(axis=0)
    gene_cells = binarized_df.sum(axis=0)

    good_cell = (
        (cell_min_counts <= cell_counts)
        & (cell_counts <= cell_max_counts)
        & (cell_min_genes <= cell_genes)
        & (cell_genes <= cell_max_genes)
    )
    good_gene = (
        (gene_min_counts <= gene_counts)
        & (gene_counts <= gene_max_counts)
        & (gene_min_cells <= gene_cells)
        & (gene_cells <= gene_max_cells)
    )
    return df.iloc[good_cell.values, good_gene.values].copy().astype("Sparse[int16]")


def normalize(sparse_df):
    X = sparse_df.sparse.to_coo()
    median = np.median(X.sum(axis=1).A.flatten())
    print(median)
    X = (preprocessing.normalize(X, norm='l1') * median).log1p().toarray()
    scaler = preprocessing.StandardScaler().fit(X)
    X = scaler.transform(X).astype('float32')
    return X, scaler.mean_, scaler.scale_  # , scaler.mean_, scaler.scale_


In [3]:
DATA_DIR = '/home/tiankang/wusuowei/data/single_cell/babel/snareseq_GSE126074/'
COUNT_FILE = 'GSE126074_AdBrainCortex_SNAREseq_cDNA.counts.mtx.gz'
CELL_FILE = 'GSE126074_AdBrainCortex_SNAREseq_cDNA.barcodes.tsv.gz'
GENE_FILE = 'GSE126074_AdBrainCortex_SNAREseq_cDNA.genes.tsv.gz'


raw_X = read_raw(DATA_DIR + COUNT_FILE, DATA_DIR + CELL_FILE, DATA_DIR + GENE_FILE)

In [18]:
X = raw_X.sparse.to_coo().tocsr()

In [24]:
X.dtype

dtype('int64')

In [63]:
scaler = preprocessing.StandardScaler(with_mean=False).fit(X)

In [60]:
def svd_with_sparse(X, k, solver='arpack', mu=None, random_state=None):
    X = X.astype('float32')
    random_init = np.random.rand(np.min(X.shape))

    if mu is None:
        mu = X.mean(axis=0).A.flatten()  # d
    mdot = mu.dot
    mmat = mdot
    mhdot = mu.T.dot
    mhmat = mu.T.dot
    Xdot = X.dot
    Xmat = Xdot
    XHdot = X.T.conj().dot
    XHmat = XHdot
    ones = np.ones(X.shape[0])[None, :].dot
    XH = X.T.conj()  # d x n

    def matvec(x):
        # print(x.shape)
        return X @ x - mu @ x
        # return Xdot(x) - mdot(x)

    def matmat(x):
        # print(x.shape)
        return X @ x - (mu @ x)[:, np.newaxis]
        # return Xmat(x) - mmat(x)

    def rmatvec(x):
        # x: n
        # print(x.shape)
        # print((XH @ x).shape)
        # print((mu * x.sum()).T.shape)
        return XH @ x - mu * x.sum()
        # return XHdot(x) - mhdot(ones(x))

    def rmatmat(x):
        # x: n * k
        return XH @ x - (mu * x.sum())[:, np.newaxis]
        # return XHmat(x) - mhmat(ones(x))

    XL = LinearOperator(
        matvec=matvec,
        matmat=matmat,
        rmatvec=rmatvec,
        rmatmat=rmatmat,
        shape=X.shape,
        dtype=X.dtype,
    )

    u, s, v = svds(XL, solver=solver, k=k, v0=random_init)
    return u, s, v

In [71]:
u, s, v = svd_with_sparse(X, k=10)

In [73]:
s

array([ 500.0534 ,  533.1472 ,  561.10144,  640.27057,  673.54956,
        841.2987 ,  919.8115 , 1050.427  , 1808.4785 , 6074.537  ],
      dtype=float32)

In [72]:
(s ** 2 / X.shape[0]).sum() / scaler.var_.sum()

0.6993347138335441

In [68]:
scaler.var_.sum()

6178.089903032298

In [5]:
qc_X = qc(raw_X, cell_min_genes=200, cell_max_genes=2500)

In [10]:
normalized_X = normalize(qc_X)

1332.0


In [61]:
X_train, X_val = train_test_split(normalized_X, test_size=0.2)

In [63]:
np.save('snare_seq_rna_profile_train.npy', X_train)
np.save('snare_seq_rna_profile_val.npy', X_val)

In [ ]:
scaler = preprocessing.StandardScaler().fit()

In [79]:
X = qc_X.sparse.to_coo()

In [91]:
qc_X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10309 entries, 09A_AAACCAACGCCT to 09L_TTTTCTATTAAG
Columns: 33160 entries, 0610005C13Rik to n-R5s50
dtypes: Sparse[int16, 0](33160)
memory usage: 56.4+ MB


In [81]:
X.data *= 2

In [92]:
X.data = np.random.randint(0, 10, 9805813)

In [93]:
X.data

array([7, 7, 7, ..., 7, 2, 4])

In [94]:
def pseudo_replicate(sparse_X, p):
    replicate_data = np.array([np.random.binomial(i, p, 2) for i in sparse_X.data])
    replicate1 = sparse_X.copy()
    replicate1.data = replicate_data[:, 0]
    replicate2 = sparse_X.copy()
    replicate2.data = replicate_data[:, 1]
    return replicate1, replicate2



In [95]:
X1, X2 = pseudo_replicate(X, 0.9)

In [97]:
X1.sum()

39716816

In [100]:
X.sum() * 0.9

39719097.0

In [101]:
preprocessing.scale([1,1,1,1,1])

array([0., 0., 0., 0., 0.])

In [1]:
np.median(X.sum(axis=1))

NameError: name 'np' is not defined